In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
from scipy.spatial.distance import cdist
import time
from tqdm import tqdm
%load_ext autoreload
%autoreload 1
#%aimport __pycache__.lddt.cpython-39
%aimport sautils.encoder_commons
%aimport sautils.common
%aimport torch
import pandas as pd
import numba
from skopt import dump, load

In [2]:
sid2seq_ss = {}
with open('data/ss.fasta') as file:
    sid = ''
    for line in file:
        line = line.lstrip('>').rstrip('\n')
        if not sid:
            sid = line
        else:
            sid2seq_ss[sid] = line.upper()
            sid = ''
sid2seq_aa = {}
with open('data/aa.fasta') as file:
    sid = ''
    for line in file:
        line = line.lstrip('>').rstrip('\n')
        if not sid:
            sid = line
        else:
            sid2seq_aa[sid] = line.upper()
            sid = ''
mat3di = np.load('data/mat3di.npy') * 2.1
blosum = np.load('data/blosum62.npy') * 1.4
with open('data/mat3di.out') as f:
    lines = f.readlines()
i2a = lines[3].split()
def seq2num(q):
    idx = []
    for a in q:
        idx.append(i2a.index(a))
    return np.array(idx)
    
def calcS_np(ss1,ss2 ,aa1, aa2,mat3di, blosum):
    ss1 = seq2num(ss1)
    ss2 = seq2num(ss2)
    aa1 = seq2num(aa1)
    aa2 = seq2num(aa2)
    comb_array_ss = np.array(np.meshgrid(ss1, ss2)).T.reshape(-1, 2)
    comb_array_aa = np.array(np.meshgrid(aa1, aa2)).T.reshape(-1, 2)
    S = (mat3di[comb_array_ss[:,0],comb_array_ss[:,1]].reshape(len(ss1),-1)) + blosum[comb_array_aa[:,0],comb_array_aa[:,1]].reshape(len(aa1),-1)
    return S
def distances(q_points_aln, t_points_aln):
    d_mat = cdist(q_points_aln, t_points_aln)
    return (d_mat)

In [3]:
coord_dir = 'data/pdb/d*'
if not os.path.exists(coord_dir):
    print('Creating cache first.')
    sautils.encoder_commons.init_coords('data/pdb/d*', coord_dir)
sid2coords = {sid: sautils.encoder_commons.read_coords(coord_dir, sid) for sid in sautils.encoder_commons.list_sids(coord_dir)}

In [4]:
@numba.jit(nopython=True)
def fill_dp_matrix(scores, gap_open, gap_extend):
    l1, l2 = scores.shape
    s, q_gap, t_gap = np.zeros((l1+1, l2+1)), np.zeros((l1+1, l2+1)), np.zeros((l1+1, l2+1))
    for i in range(1, l1+1):
        for j in range(1, l2+1):
            q_gap[i, j] = max(q_gap[i, j-1] - gap_extend, s[i, j-1] - gap_open)
            t_gap[i, j] = max(t_gap[i-1, j] - gap_extend, s[i-1, j] - gap_open)
            s[i, j] = max(0,
                          q_gap[i, j],
                          t_gap[i, j],
                          s[i-1, j-1] + scores[i-1, j-1])
    return s, q_gap, t_gap

def backtrace_alignment(scores, gap_open, gap_extend, s, q_gap, t_gap):
    S_MAT, QGAP_MAT, TGAP_MAT = 0, 1, 2
    arg_max = np.where(s == s.max())
    i, j, pos = arg_max[0][0], arg_max[1][0], S_MAT
    backtrace = []
    while (s[i, j] > 0 or pos != S_MAT) and (i > 0 and j > 0):
        if pos == S_MAT:
            if s[i, j] == q_gap[i, j]:
                if q_gap[i,j] == q_gap[i, j-1] - gap_extend:
                    pos = QGAP_MAT
                j = j - 1       
            elif s[i, j] == t_gap[i, j]:
                if t_gap[i,j] == t_gap[i-1, j] - gap_extend:
                    pos = TGAP_MAT
                i = i - 1
            elif s[i,j] == s[i-1, j-1] + scores[i-1, j-1]:
                backtrace.append((i,j))
                i, j = i - 1, j - 1                
            else:
                break        
        elif pos == QGAP_MAT:
            if q_gap[i, j] == s[i, j-1] - gap_open:
                pos = S_MAT
            j = j - 1
            
        elif pos == TGAP_MAT:
            if t_gap[i, j] == s[i-1, j] - gap_open:
                pos = S_MAT
            i = i - 1
    return list(reversed([(i-1, j-1) for i, j in backtrace]))

def sw_align(scores, gap_open=30, gap_extend=10):    
    s, q_gap, t_gap = fill_dp_matrix(scores, gap_open, gap_extend)
    backtrace = backtrace_alignment(scores, gap_open, gap_extend, s, q_gap, t_gap)
    return backtrace, (s, q_gap, t_gap)

def format_aln(seq1, seq2, bt):
    s1, s2 = '', ''
    i_prev, j_prev = None, None
    for i, j in bt:        
        if i_prev is not None:
            s1 += seq1[i_prev+1:i] + '-' * len(seq2[j_prev+1:j])
            s2 += '-' * len(seq1[i_prev+1:i]) + seq2[j_prev+1:j]                
        s1 += seq1[i]
        s2 += seq2[j]
        i_prev, j_prev = i, j
    return s1, s2

In [5]:
def fb_local(S, go, ge, T, pre):
    S_exp = np.exp(S/T, dtype = pre)
    S_hat = S_exp[::-1,::-1]

    vj = np.exp(np.linspace((S.shape[1]-1)*ge + go, go, S.shape[1], dtype = pre)/T, dtype = pre)
    wj = np.exp(np.linspace((S.shape[1]-1)*ge , 0, S.shape[1], dtype = pre) /T, dtype = pre)
    #vj = np.exp(np.arange((S.shape[1]-1)*ge + go ,go-ge, -ge, dtype = pre)/T)
    #wj = np.exp((np.arange((S.shape[1]-1)*ge , -ge, -ge, dtype = pre)) /T)
    zm, ze, zf = forward_backward_plain_local(S_exp, vj, wj, T, go, ge, pre)
    zm_hat, ze_hat, zf_hat = forward_backward_plain_local(S_hat, vj, wj, T, go, ge, pre)

    P = ((zm*(zm_hat[::-1,::-1]))/(1 + np.sum(zm))) * np.exp(-S/T, dtype = pre)
    
    return P
    
def fb_global(S, go, ge, T, pre):
    S_exp = np.exp(S/T, dtype = pre)
    S_hat = S_exp[::-1,::-1]

    vj = np.exp(np.linspace((S.shape[1]-1)*ge + go, go, S.shape[1])/T, dtype = pre)
    wj = np.exp(np.linspace((S.shape[1]-1)*ge , 0, S.shape[1]) /T, dtype = pre)
    zm, ze, zf = forward_backward_plain_global(S_exp, vj, wj, T, go, ge, pre)
    zm_hat, ze_hat, zf_hat = forward_backward_plain_global(S_hat, vj, wj, T, go, ge, pre)

    P = (zm*(zm_hat[::-1,::-1]))/ (zm[-1,-1]+ze[-1,-1] +zf[-1,-1]) * np.exp(-S/T, dtype = pre)
    
    return P
def fb_local_global(S, go, ge, T, pre):
    S_exp = np.exp(S/T, dtype = pre)
    S_hat = S_exp[::-1,::-1]

    vj = np.exp(np.linspace((S.shape[1]-1)*ge + go, go, S.shape[1])/T, dtype = pre)
    wj = np.exp(np.linspace((S.shape[1]-1)*ge , 0, S.shape[1]) /T, dtype = pre)
    zm, ze, zf = forward_backward_plain_local(S_exp, vj, wj, T, go, ge, pre)
    zm_hat, ze_hat, zf_hat = forward_backward_plain_global(S_hat, vj, wj, T, go, ge, pre)

    P = (zm*(zm_hat[::-1,::-1]))/ (zm[-1,-1]+ze[-1,-1] +zf[-1,-1])* np.exp(-S/T, dtype = pre)
    
    return P
    
def fb_global_local(S, go, ge, T, pre):
    S_exp = np.exp(S/T, dtype = pre)
    S_hat = S_exp[::-1,::-1]

    vj = np.exp(np.linspace((S.shape[1]-1)*ge + go, go, S.shape[1])/T, dtype = pre)
    wj = np.exp(np.linspace((S.shape[1]-1)*ge , 0, S.shape[1]) /T, dtype = pre)
    zm, ze, zf = forward_backward_plain_global(S_exp, vj, wj, T, go, ge, pre)
    zm_hat, ze_hat, zf_hat = forward_backward_plain_local(S_hat, vj, wj, T, go, ge, pre)

    P = (zm*(zm_hat[::-1,::-1]))/ np.sum(zm) * np.exp(-S/T)
    
    return P

In [6]:
def forward_backward_plain_local(S, vj, wj, T, go, ge, pre):
    exp_go = np.exp(go/T)
    exp_ge = np.exp(ge/T)
    zm = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    ze = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    zf = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    
    #This is the main part where we calculate the Z matrices with one for loop over each row
    for i in range(1,S.shape[0]+1):
        zm[i,1:] = ((zm[i-1,:-1] + ze[i-1,:-1] + zf[i-1,:-1] + 1) *S[i-1,:])
        # this is the new part where ze is calculated using cumsum 
        ze[i,1:] = (np.cumsum(zm[i,:-1] * vj)/wj) 
        zf[i,1:] = (zm[i-1,1:] * exp_go + zf[i-1,1:] * exp_ge)
    #at the end we only need to return zm but for testing i returned all
    return zm[1:,1:], ze[1:,1:], zf[1:,1:]

def forward_backward_plain_global(S, vj, wj, T, go, ge, pre):
    exp_go = np.exp(go/T)
    exp_ge = np.exp(ge/T)
    zm = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    ze = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    zf = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    
    zm[0,0] = 1
    ze[0,1:] = np.exp(np.linspace(go,go+(ge*(S.shape[1]-1)),S.shape[1])/T, dtype = pre)

    #exp_ge_arr = np.exp((np.arange(ge, (S.shape[1] + 1)* ge ,ge,dtype = pre))/T, dtype = pre)
                        
    for i in range(1,S.shape[0]+1):
        zm[i,1:] = (zm[i-1, :-1] + ze[i-1,:-1] + zf[i-1,:-1]) * S[i-1,:]
        ze[i,1:] = (np.cumsum(zm[i,:-1] * vj)/wj) #+  * exp_ge_arr
        zf[i,:] = zm[i-1,:] * exp_go + zf[i-1,:] * exp_ge
    return zm[1:,1:], ze[1:,1:], zf[1:,1:]

In [7]:
def forward_backward_plain_local(S, vj, wj, T, go, ge, pre):
    exp_go = np.exp(go/T)
    exp_ge = np.exp(ge/T)
    zm = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    ze = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    zf = np.zeros((S.shape[0]+1, S.shape[1]+1), dtype = pre)
    
    #This is the main part where we calculate the Z matrices with one for loop over each row
    for i in range(1,S.shape[0]+1):
        zm[i,1:] = ((zm[i-1,:-1] + ze[i-1,:-1] + zf[i-1,:-1] + 1) *S[i-1,:])
        # this is the new part where ze is calculated using cumsum 
        ze[i,1:] = (np.cumsum(zm[i,:-1] * vj)/wj) 
        zf[i,1:] = (zm[i-1,1:] * exp_go + zf[i-1,1:] * exp_ge)
    #at the end we only need to return zm but for testing i returned all
    return zm[1:,1:], ze[1:,1:], zf[1:,1:]
def fb_local(S, go, ge, T, pre):
    S_exp = np.exp(S/T, dtype = pre)
    S_hat = S_exp[::-1,::-1]

    vj = np.exp(np.linspace((S.shape[1]-1)*ge + go, go, S.shape[1], dtype = pre)/T, dtype = pre)
    wj = np.exp(np.linspace((S.shape[1]-1)*ge , 0, S.shape[1], dtype = pre) /T, dtype = pre)
    #vj = np.exp(np.arange((S.shape[1]-1)*ge + go ,go-ge, -ge, dtype = pre)/T)
    #wj = np.exp((np.arange((S.shape[1]-1)*ge , -ge, -ge, dtype = pre)) /T)
    zm, ze, zf = forward_backward_plain_local(S_exp, vj, wj, T, go, ge, pre)
    zm_hat, ze_hat, zf_hat = forward_backward_plain_local(S_hat, vj, wj, T, go, ge, pre)

    P = ((zm*(zm_hat[::-1,::-1]))/(1 + np.sum(zm))) * np.exp(-S/T, dtype = pre)
    
    return P

In [8]:
@numba.njit
def lol_score_approx(d_ij, d_kl, d_seq):
    seq_distance_feat = np.sign(d_seq) * np.log(1 + np.abs(d_seq))
    x = np.empty((d_ij.shape[0],2))
    x[:,0] = seq_distance_feat#[:, None]
    x[:,1] = np.abs(d_ij-d_kl)#[:, None]
    #x = np.hstack([seq_distance_feat[:, None], np.abs(d_ij-d_kl)[:, None],])

    w1 = np.array([[-1.3584513e-04,  9.9329501e-01],
                   [ 7.6149112e-01,  5.7029408e-01],
                   [-8.1348085e-01,  6.0702705e-01]])
    b1 = np.array([0.7043129 , 0.374659  , 0.39905924])
    w2 = np.array([[-0.776632  ,  0.61055756,  0.5823986 ]])
    b2 = np.array([-0.11200039])

    hidden_layer = ((x @ w1.T) + b1)
    hidden_layer *= (hidden_layer > 0)  # if w1 is all positive, we dont need this ReLU
    score = (hidden_layer @ w2.T) + b2
    return score

In [9]:
def calc_startAnchors(S, fb_go, fb_ge, temp, le):
    start_anchors = []
    for i in range(2):
        P = fb_local(S, fb_go, fb_ge, temp, np.longdouble)
        x,y = np.unravel_index(np.argmax(P), P.shape)
        x,y = np.max([0,x-le]), np.max([0,y-le])
        #P -= np.max([0 ,np.max(P)-shift])
        bt, (s, q_gap, t_gap) = sw_align(np.array(P[x:x+(le*2),y:y+(le*2)], dtype = np.double), gap_open = 0, gap_extend=0)
        bt = np.array(bt)
        bt[:,0] += x
        bt[:,1] += y
        start_anchors.append(np.copy(bt))
        S[bt[:,0], bt[:,1]] = - 100
    return start_anchors

In [10]:
def get_gaps(A_all):
    lowerBounds_i = (A_all[:,0]+1)[:-1]
    upperBounds_i = (A_all[:,0]-1)[1:]
    lowerBounds_k = (A_all[:,1]+1)[:-1]
    upperBounds_k = (A_all[:,1]-1)[1:]
    mask_i = lowerBounds_i<=upperBounds_i
    mask_k = lowerBounds_k<=upperBounds_k
    mask = np.logical_and(mask_i,  mask_k)
    return np.array([np.sort(np.array([lowerBounds_i[mask],upperBounds_i[mask]+1]).reshape(-1)),
                    np.sort(np.array([lowerBounds_k[mask],upperBounds_k[mask]+1]).reshape(-1))])

In [11]:

def lol_matrix_gap_dist_aprox(A, All_a, gaps,d_mat_ij, d_mat_kl):

    ti, tk, tidx = lol_matrix_prep_gaps(A, All_a, gaps, d_mat_ij, d_mat_kl)
    idx = np.logical_and(ti > 0, tk > 0)
    G_mat = np.zeros_like(ti)
    G_mat[idx] += np.squeeze(np.nan_to_num(lol_score_approx(ti[idx], tk[idx], tidx[idx])))
    return G_mat.reshape(gaps[0,1]-gaps[0,0], gaps[1,1]-gaps[1,0], -1).sum(axis=2)

In [12]:
def lol_matrix_prep_gaps(A, All_a, gaps,d_mat_ij, d_mat_kl):
    #print(d_mat_ij[:,gaps[0,0]:gaps[0,1]].shape, d_mat_kl[:,gaps[1,0]:gaps[1,1]].shape)
    d_ij, d_kl = np.array(d_mat_ij[A[:,0],gaps[0,0]:gaps[0,1]]), np.array(d_mat_kl[A[:,1],gaps[1,0]:gaps[1,1]])
    ti = np.repeat(d_ij.T, gaps[1,1] - gaps[1,0], axis = 0).reshape(-1)
    tk = np.repeat(d_kl.T.reshape(-1,1), gaps[0,1]-gaps[0,0], axis = 1).T.reshape(-1)
    tidx = np.repeat(A[:,0].reshape(1,-1), d_ij.shape[1] * d_kl.shape[1], axis = 0).reshape(-1)
    #print(ti.shape, tk.shape, tidx.shape, np.repeat(np.arange(gaps[0,0],gaps[0,1]), (gaps[1,1]-gaps[1,0]) * A.shape[0]).shape)
    tidx = tidx - np.repeat(np.arange(gaps[0,0],gaps[0,1]), (gaps[1,1]-gaps[1,0]) * A.shape[0])

    return ti, tk, tidx

In [13]:
def align(S, startAnchors, go, ge, min_m, red_m, temp, red_t, d_mat_ij, d_mat_kl):
    S_all = []
    ap_all = []
    pre = np.longdouble
    lol_matrix_prep = lol_matrix_prep_gaps
    for j in range(2):
        num_all = 0
        th = .99
        tempr = temp
        S = np.zeros_like(S, dtype = pre)
        A_new = startAnchors[j]
        A_all = startAnchors[j]

        for i in (range(1000)):
            #S[A_all[:,0], A_all[:,1]] = 1e2
            gaps_idn = get_gaps(np.unique(np.concatenate((np.array([-1,-1]).reshape(1,2), A_all, np.array([S.shape[0], S.shape[1]]).reshape(1,2))), axis = 0))
            P = np.zeros_like(S)
            
            #for g in range(0, gaps_idn.shape[1], 2):
            #    S[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]] += lol_matrix_gap_dist_aprox(A_new, A_all, gaps_idn[:,g:g+2] ,d_mat_ij, d_mat_kl)
            S += lol_matrix_gap_dist_aprox(A_new, A_all, np.array([[0,0],[S.shape[0], S.shape[1]]]).T ,d_mat_ij, d_mat_kl)
            for g in range(0, gaps_idn.shape[1], 2):
                P[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]] = fb_global(S[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]], go, ge, tempr, pre)
                if (g == 0 and gaps_idn[0,g] == 0 and gaps_idn[1,g] == 0):
                    P[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]] = fb_local_global(S[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]], go, ge, tempr, pre)
                elif(g == (gaps_idn.shape[1]-2), gaps_idn[0,g+1] == S.shape[0], gaps_idn[1,g+1] == S.shape[1]):
                    P[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]] = fb_local_global(S[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]], go, ge, tempr, pre)
                    
                else:
                    P[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]] = fb_global(S[gaps_idn[0,g]:gaps_idn[0,g+1],gaps_idn[1,g]:gaps_idn[1,g+1]], go, ge, tempr, pre)
                    

            m = np.max(P)
            if m <= min_m:
                break
            A_new = np.argwhere(P >= np.max([m - red_m, min_m]))
            A_all = np.vstack((A_all, A_new))
            A_all = np.sort(A_all, axis = 0)
            #num_all -= 1
            #tempr = np.max([tempr-red_t, 1])
        #S[A_all[:,0], A_all[:,1]] = 1e2
        S_all.append(np.copy(S))
        ap_all.append(np.copy(A_all))
    return S_all, ap_all

In [42]:
class lol_alignment():
    def __init__(self, sa_fb_go=-3, sa_fb_ge=-2, sa_temp = 1, le = 3,
                 lol_go=-3, lol_ge=0, lol_min_m=0.4,  lol_red_t = 0, lol_temp=2):
        self.sa_fb_go = sa_fb_go
        self.sa_fb_ge = sa_fb_ge
        self.sa_temp = sa_temp
        self.le = le
        self.lol_go = lol_go
        self.lol_ge = lol_ge 
        self.lol_min_m = lol_min_m
        self.lol_red_m = 0.1
        self.lol_red_t = lol_red_t
        self.lol_temp = lol_temp
        self.rescale_score = {}
        self.di_score = {}
        self.alignments = {}
        self.lol_mat = {}

    def fit(self, querys, targets,*kwargs):
        for query,target in tqdm(zip(querys, targets), total = targets.shape[0]):
            coords1,_ = sid2coords.get(query)
            coords2,_ = sid2coords.get(target)
            coords1 = coords1[:,:3]
            coords2 = coords2[:,:3]
            d_mat_ij = distances(coords1, coords1)
            d_mat_ij[d_mat_ij > 15] = 0
            d_mat_kl = distances(coords2, coords2)
            d_mat_ij = np.asarray(d_mat_ij)
            d_mat_kl = np.asarray(d_mat_kl)
            aa1 = sid2seq_aa[query]
            aa2 = sid2seq_aa[target]
            ss1 = sid2seq_ss[query]
            ss2 = sid2seq_ss[target]
            S = calcS_np(ss1, ss2, aa1, aa2, mat3di, blosum)
            st = time.time()
            startAnchors = calc_startAnchors(np.copy(S), self.sa_fb_go, self.sa_fb_ge, self.sa_temp, self.le)
            S_lols, aps = align(S, startAnchors, self.lol_go, self.lol_ge, self.lol_min_m,
                                                    self.lol_red_m, self.lol_temp, self.lol_red_t, d_mat_ij, d_mat_kl)
            #self.alignments[query+"_"+target] = align(S, startAnchors, self.lol_go, self.lol_ge, self.lol_min_m,
            #                                       self.lol_red_m, self.lol_temp, self.lol_red_t, d_mat_ij, d_mat_kl)
            rescale_scores = []
            di_scores = []
            #print(time.time() - st, S.shape[0], S.shape[1])
            for i in range(2):
                self.alignments[query+"_"+target+'_'+str(i)] = aps[i]
                self.lol_mat[query+"_"+target+'_'+str(i)] = S_lols[i]
                S_lol, ap = S_lols[i], aps[i]
                rescale_scores.append(((S+S_lol) * fb_local(S +S_lol, -3.5, -.3, 2, np.longdouble)).sum() / np.sqrt(S.shape[0]*S.shape[1]))
                di_scores.append(((S+S_lol) * fb_local(S + S_lol, -30, -10, 15, np.longdouble)).sum())
            self.rescale_score[query+"_"+target] = np.max(rescale_scores)
            self.di_score[query+"_"+target] = np.max(di_scores)
            #print(time.time() - st, S.shape[0], S.shape[1])
            


In [43]:
test_align = lol_alignment()

In [32]:
query, target = "d2v33a_", "d2v33a_"
test_align2.fit([query],[target])

[5668.6676934627865796, 5668.6676934627865885] [62.356341380184142107, 62.356341380184142173]


In [49]:
test_align.di_score['d2v33a__d1ayoa_']

1464.9271906243007202

In [44]:
%%time
test_align.fit(querys,targets)

100%|█████████████████████████████████████| 11211/11211 [44:07<00:00,  4.24it/s]

CPU times: user 2h 48min 35s, sys: 7min 48s, total: 2h 56min 23s
Wall time: 44min 7s


In [64]:
test_align.fit(np.array(['d1h97a_']),np.array(['d1jjcb3']))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.08it/s]

(147, 113)


In [47]:
all_cl = pd.read_csv('100vsall/data/scop_lookup.fix.tsv', delimiter='\t', header=None)

In [48]:
all_cl

,0,1
0,d1dlwa_,a.1.1.1
1,d2gkma_,a.1.1.1
2,d2qrwa_,a.1.1.1
3,d1s69a_,a.1.1.1
4,d2bkma_,a.1.1.1
...,...,...
11206,d2fk4a1,g.90.1.1
11207,d2b9da1,g.91.1.1
11208,d2ewla1,g.91.1.1
11209,d2pkgc1,g.92.1.1


In [52]:
cl

['b.1.18.2',
 'c.55.1.5',
 'd.110.2.0',
 'a.60.7.1',
 'b.85.4.1',
 'b.36.1.1',
 'a.4.5.10',
 'd.142.1.2',
 'c.69.1.20',
 'd.157.1.1',
 'f.4.3.1',
 'g.19.1.1',
 'b.12.1.2',
 'a.275.1.1',
 'd.282.1.1',
 'c.90.1.1',
 'c.2.1.6',
 'd.67.1.1',
 'a.43.1.3',
 'b.18.1.2',
 'd.180.1.1',
 'b.84.1.0',
 'a.4.5.4',
 'a.60.12.1',
 'c.69.1.29',
 'd.17.4.3',
 'g.25.1.1',
 'c.1.7.1',
 'd.32.1.8',
 'g.39.1.16',
 'd.73.1.1',
 'c.2.1.7',
 'd.15.14.1',
 'c.1.23.2',
 'b.33.1.1',
 'd.50.1.1',
 'a.274.1.1',
 'b.34.5.3',
 'b.41.1.3',
 'c.26.1.1',
 'd.144.1.7',
 'd.58.1.5',
 'd.20.1.1',
 'd.159.1.7',
 'a.144.1.1',
 'b.1.1.1',
 'd.17.1.1',
 'b.68.10.1',
 'c.2.1.2',
 'd.21.1.2',
 'b.36.1.1',
 'c.1.4.1',
 'd.26.1.1',
 'c.47.1.5',
 'b.3.2.1',
 'c.95.1.2',
 'c.37.1.20',
 'd.93.1.1',
 'c.1.17.2',
 'a.69.3.1',
 'a.156.1.2',
 'a.4.5.83',
 'b.40.4.5',
 'd.268.1.4',
 'c.3.1.5',
 'b.47.1.2',
 'd.58.14.1',
 'd.359.1.1',
 'f.26.1.1',
 'c.17.1.1',
 'c.37.1.1',
 'd.169.1.1',
 'a.4.13.2',
 'b.21.1.3',
 'd.167.1.1',
 'c.1.12.5',

In [51]:
cl = []
for t in targets:
    cl.append(all_cl[all_cl[0].values == t][1].values[0])

In [26]:
di_scores

[{'d2v33a__d1k3ia1': 1017.6521487874883538,
  'd2v33a__d1zbsa2': 271.82909132264064575,
  'd2v33a__d3mmha_': 93.649317505785797694,
  'd2v33a__d1tfra1': -21.15892499010842805,
  'd2v33a__d3hzaa_': 736.37066579603200184,
  'd2v33a__d1v62a_': 180.79822287106331613,
  'd2v33a__d2nrac1': 86.34628430158087305,
  'd2v33a__d1kjqa3': 76.199994216264615055,
  'd2v33a__d3c70a_': 71.405117403605207954,
  'd2v33a__d1ko3a_': 223.3729145275411097,
  'd2v33a__d3prna_': 75.275470410012751625,
  'd2v33a__d1beia_': -10.789874983858839461,
  'd2v33a__d1bu8a1': 83.48095863453035176,
  'd2v33a__d2i5ua1': -22.230942877707147762,
  'd2v33a__d1tlja_': 65.743646343893829824,
  'd2v33a__d1pjqa2': 63.114347133760181015,
  'd2v33a__d1qmga2': -15.7768437394510901185,
  'd2v33a__d1tkea2': 6.043990954096862756,
  'd2v33a__d2bj7a1': -14.6358793080717767415,
  'd2v33a__d2qqia1': 427.23318783932743575,
  'd2v33a__d16vpa_': 20.087039642428744322,
  'd2v33a__d3klra_': 126.52438065693251883,
  'd2v33a__d2zcwa1': 129.71681

In [25]:
rescale_scores=[test_align.rescale_score]
di_scores=[test_align.di_score]

In [ ]:
%%time
for i in range(50):
    querys = np.repeat(alignments[0].values[i+1], 11211)
    #test_align = lol_alignment()
    test_align.fit(querys,targets)
    rescale_scores.append(test_align.rescale_score)
    di_scores.append(test_align.di_score)

In [59]:
test_align.di_score.values()

dict_values([917.8558246101764465])

In [79]:
t =[]
for k in test_align.di_score:
    t.append(k[8:])

In [56]:
df = pd.DataFrame([querys,targets, test_align.rescale_score.values(), cl]).T

In [57]:
df = df.sort_values([2], ascending=[False])
df

,0,1,2,3
4788,d2v33a_,d2v33a_,62.356341,b.1.18.4
3149,d2v33a_,d1f00i1,23.938701,b.1.14.1
779,d2v33a_,d1pbya3,22.958673,b.1.18.14
7413,d2v33a_,d1cwva1,22.832755,b.1.14.1
1556,d2v33a_,d1l9na3,22.789691,b.1.5.1
...,...,...,...,...
3802,d2v33a_,d1w3ba_,0.273545,a.118.8.1
1538,d2v33a_,d2clya1,0.263334,f.52.1.1
7143,d2v33a_,d1q90m_,0.244777,f.23.25.1
5940,d2v33a_,d1l2pa_,0.236548,f.23.21.1


In [58]:
df.to_csv("100vsall/test_rescale3.csv", sep='\t', header=False, index = False)

In [29]:
alignments[0].values[1]


'd1h97a_'

In [34]:
listing = os.listdir('data/pdb/d*')
targets = []
for infile in listing:
    targets.append(infile[:7])

In [35]:
querys = np.repeat(['d2v33a_'], len(targets))

In [36]:
targets = np.array(targets)

In [37]:
querys, targets

(array(['d2v33a_', 'd2v33a_', 'd2v33a_', ..., 'd2v33a_', 'd2v33a_',
        'd2v33a_'], dtype='<U7'),
 array(['d1k3ia1', 'd1zbsa2', 'd3mmha_', ..., 'd3fnca_', 'd1v25a_',
        'd1ow1a_'], dtype='<U7'))

In [96]:
querys = pd.read_csv("100vsall/querys.csv", sep='\t', header=None)[0].values
querys

array(['d2jhfa1', 'd2huga1', 'd1ivha1', 'd1jyaa_', 'd1hyua4', 'd2bz7a_',
       'd2dfka2', 'd1qpoa2', 'd1rowa_', 'd1g2oa_', 'd2bwba_', 'd1vd4a_',
       'd1vq8n1', 'd3n9ka_', 'd1l1oc_', 'd1alla_', 'd1th7a1', 'd2ifga1',
       'd1vhwa_', 'd2nqca1', 'd1ugla_', 'd1uw4b_', 'd2y8ga_', 'd2cqka1',
       'd2cw9a1', 'd3ersx_', 'd2pl1a_', 'd1urra_', 'd1h6ka1', 'd1kcqa_',
       'd1gpma1', 'd1cqxa1', 'd1nvma2', 'd2q7wa_', 'd1yk4a_', 'd2bhua1',
       'd1o5za2', 'd1k92a1', 'd1rwha1', 'd3pt8a_', 'd1ckaa_', 'd2imla1',
       'd1k8ua_', 'd1p6oa_', 'd1jjcb3', 'd2v33a_', 'd2nuha_', 'd2bs2b2',
       'd1p7hl2', 'd1ayja_', 'd3ljqa_', 'd1e0ba_', 'd2adca1', 'd2f4pa1',
       'd1l2ha_', 'd1sv0c_', 'd1v5oa_', 'd1rkta1', 'd2cpwa1', 'd2bkfa1',
       'd2hq7a1', 'd2b6ca1', 'd1twia1', 'd2jxca1', 'd1vc1a_', 'd1ixra1',
       'd3edha_', 'd1bf6a_', 'd3bw6a_', 'd1qp8a2', 'd1bupa1', 'd2cpia1',
       'd1t9ma_', 'd1m6ba1', 'd2fvka2', 'd2b5ea1', 'd1m0ua2', 'd2fhfa5',
       'd1wfqa_', 'd2vaca_', 'd2dk5a1', 'd1ypzf1', 

In [98]:
program_starts = 0
for query in querys[program_starts:program_starts+20]:
    print(query)

d2jhfa1
d2huga1
d1ivha1
d1jyaa_
d1hyua4
d2bz7a_
d2dfka2
d1qpoa2
d1rowa_
d1g2oa_
d2bwba_
d1vd4a_
d1vq8n1
d3n9ka_
d1l1oc_
d1alla_
d1th7a1
d2ifga1
d1vhwa_
d2nqca1
